In [2]:
from matplotlib import pyplot
import matplotlib.pyplot as plt

import torch

import torch.nn.functional as F
import torch.optim as optim

from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset

import numpy as np
import pandas as pd

import math
import cv2
import re

In [3]:
files = [open('all_train.txt', 'r'), open('all_test1.txt', 'r'), open('all_test2.txt', 'r')]

In [4]:
lable_dic = {'straight' : 0, 'up':1, 'left': 2, 'right': 3}

X_l = []
y_l = []
for file in files:
    paths = []
    y = []
    for line in file.readlines():
        path = '.'
        for s in line[:-1].split('/')[-3:]:
            path += '/'+ s
        paths.append(path)

        lable = line.split('_')[1]
        y.append(lable_dic[lable])

    x = []
    for path in paths:
        img = cv2.imread(path, 0).flatten()
        x.append(img)
    x = np.array(x)
    
    X_l.append(x)
    y_l.append(y)

In [5]:
X_t_l = []
y_t_l = []
for X,y in zip(X_l,y_l):
    X_t, y_t = map(torch.tensor, (X, y))
    X_t = X_t.type(torch.float)
    X_t = X_t/255
    y_t = y_t.type(torch.long)
    
    X_t_l.append(X_t)
    y_t_l.append(y_t)

In [6]:
x_train, x_test_1, x_test_2 = X_t_l
y_train, y_test_1, y_test_2 = y_t_l

In [7]:
train = TensorDataset(x_train, y_train)

In [8]:
train_loader = DataLoader(train, batch_size=120, shuffle=True)

In [9]:
class Net(nn.Module):
    def __init__(self, unit=3):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(960,unit)
        self.fc2 = nn.Linear(unit,4)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return F.sigmoid(x)
    
model = Net()

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.3, momentum = 0.3)

hidden_weight_l = []
for epoch in range(100):
    total_loss = 0
    
    for train_x, train_y in train_loader:
        hidden_weight_l.append([float(i[0]) for i in model.fc2.weight])
        # 계산 그래프 구성
        train_x, train_y = Variable(train_x), Variable(train_y)
        # 경사 초기화
        optimizer.zero_grad()
        # 순전파 계산
        output = model(train_x)
        # 오차계산
        loss = criterion(output, train_y)
        # 역전파 계산
        loss.backward()
        # 가중치 업데이트
        optimizer.step()
        # 누적 오차 계산
        total_loss += loss.data
        
    if (epoch+1)%10 == 0:
        print(epoch+1, total_loss)

/Users/hyunwoo/anaconda3/envs/optlab/lib/python3.7/site-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


10 tensor(3.8572)
20 tensor(3.5676)
30 tensor(3.3242)
40 tensor(3.1471)
50 tensor(3.0808)
60 tensor(2.9861)
70 tensor(2.9186)
80 tensor(2.8729)
90 tensor(2.8417)
100 tensor(2.8169)


In [11]:
x_test_1, y_test_1 = Variable(x_test_1), Variable(y_test_1)

In [12]:
result = torch.max(model(x_test_1).data, 1)[1]

In [13]:
x_test_1.shape

torch.Size([139, 960])

In [14]:
result

tensor([0, 2, 2, 3, 3, 0, 2, 0, 0, 0, 3, 2, 0, 0, 0, 0, 3, 3, 0, 0, 2, 0, 0, 2,
        3, 0, 2, 0, 0, 2, 3, 2, 0, 0, 3, 0, 3, 3, 0, 2, 0, 3, 2, 0, 2, 0, 0, 2,
        0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 0, 0, 3, 2, 2, 0, 0, 3, 0,
        3, 0, 3, 2, 0, 0, 0, 3, 3, 2, 0, 0, 1, 3, 0, 0, 0, 0, 3, 2, 3, 2, 3, 0,
        0, 2, 3, 0, 2, 0, 2, 3, 0, 2, 2, 2, 2, 2, 3, 3, 0, 3, 2, 2, 0, 3, 2, 0,
        0, 2, 0, 0, 2, 0, 2, 0, 2, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0])

In [15]:
y_test_1

tensor([1, 1, 2, 3, 3, 0, 2, 0, 0, 1, 3, 2, 0, 0, 0, 1, 3, 1, 0, 0, 2, 1, 1, 2,
        3, 1, 2, 0, 0, 2, 3, 1, 1, 0, 1, 0, 3, 3, 1, 2, 0, 3, 2, 0, 1, 1, 1, 2,
        0, 2, 0, 1, 1, 1, 0, 0, 0, 1, 2, 0, 1, 1, 3, 0, 1, 3, 2, 2, 1, 1, 3, 1,
        3, 1, 3, 2, 0, 1, 0, 3, 3, 2, 1, 1, 0, 3, 0, 1, 1, 0, 3, 2, 3, 2, 3, 1,
        1, 2, 3, 0, 2, 0, 2, 3, 0, 2, 2, 2, 2, 2, 3, 3, 0, 3, 2, 2, 1, 3, 2, 0,
        1, 2, 0, 0, 2, 1, 2, 0, 2, 0, 3, 0, 1, 0, 1, 0, 1, 0, 1])

# Assignment 1

In [16]:
accuracy = sum(y_test_1.data.numpy() == result.numpy()) / len(y_test_1.data.numpy())

In [17]:
accuracy

0.6906474820143885

# Assignment 2

In [ ]:
for i in range(4):
    plt.plot([step[i] for step in hidden_weight_l], label=f'{list(lable_dic.keys())[i]}')
    
plt.title('Hidden layer weight')
plt.ylabel('weight')
plt.xlabel('epoch')
pyplot.legend(loc='upper right', bbox_to_anchor=(1.3, 1))

plt.show()

# Assignment 3

In [18]:
model_30 = Net(30)

In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_30.parameters(), lr=0.3, momentum = 0.3)

for epoch in range(100):
    total_loss = 0
    
    for train_x, train_y in train_loader:
        # 계산 그래프 구성
        train_x, train_y = Variable(train_x), Variable(train_y)
        # 경사 초기화
        optimizer.zero_grad()
        # 순전파 계산
        output = model_30(train_x)
        # 오차계산
        loss = criterion(output, train_y)
        # 역전파 계산
        loss.backward()
        # 가중치 업데이트
        optimizer.step()
        # 누적 오차 계산
        total_loss += loss.data
        
    if (epoch+1)%10 == 0:
        print(epoch+1, total_loss)

10 tensor(3.8807)
20 tensor(3.4312)
30 tensor(3.1404)
40 tensor(2.9516)
50 tensor(2.8137)
60 tensor(2.7023)
70 tensor(2.6261)
80 tensor(2.5952)
90 tensor(2.5533)
100 tensor(2.5154)


In [20]:
x_test_1, y_test_1 = Variable(x_test_1), Variable(y_test_1)

In [21]:
result = torch.max(model_30(x_test_1).data, 1)[1]

In [22]:
accuracy = sum(y_test_1.data.numpy() == result.numpy()) / len(y_test_1.data.numpy())

In [23]:
accuracy

0.8705035971223022